In [1]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.sql.functions.{col}
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.ml.classification.{GBTClassifier, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier, NaiveBayes, LinearSVC, LogisticRegression, OneVsRest}
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator,BinaryClassificationEvaluator}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.linalg.Vectors
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
147,application_1576053140655_0148,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.sql.functions.col
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{GBTClassifier, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier, NaiveBayes, LinearSVC, LogisticRegression, OneVsRest}
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.linalg.Vectors
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar


In [2]:
val raw = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").csv("datasets/creditcard.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

raw: org.apache.spark.sql.DataFrame = [Time: string, V1: string ... 29 more fields]


In [3]:
// cast all the column to Double type.
val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 29 more fields]


In [4]:
// convert the label from {0, 1} to {1, 2}
//val labelConverter = new FuncTransformer(udf {d: Double => if (d==0) 2 else d }).setInputCol("Class").setOutputCol("Class")
val labelConverter = new StringIndexer().setInputCol("Class").setOutputCol("label")
val assembler = new VectorAssembler().setInputCols(Array("V3", "V4", "V9", "V10", "V11", "V12", "V14", "V16", "V17", "V18","V19")).setOutputCol("assembled")
val scaler = new MinMaxScaler().setInputCol("assembled").setOutputCol("features")
//val scaler = new StandardScaler().setInputCol("assembled").setOutputCol("features")
val pipeline = new Pipeline().setStages(Array(assembler, scaler, labelConverter))
val pipelineModel = pipeline.fit(df)
val data = pipelineModel.transform(df)
println("Generate feature from raw data:")
data.select("features", "label").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelConverter: org.apache.spark.ml.feature.StringIndexer = strIdx_2c0ebbeb9c7a
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_069a5eb16c49
scaler: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_e6eabddd89ba
pipeline: org.apache.spark.ml.Pipeline = pipeline_b8aaa57c1913
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_b8aaa57c1913
data: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 32 more fields]
Generate feature from raw data:
+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.88136490328633...|  0.0|
|[0.84029849039390...|  0.0|
|[0.86814081926190...|  0.0|
|[0.86848364774806...|  0.0|
|[0.86425070140685...|  0.0|
|[0.85718742663956...|  0.0|
|[0.83819983804773...|  0.0|
|[0.85603110871019...|  0.0|
|[0.83545216735689...|  0.0|
|[0.85551101187934...|  0.0|
|[0.85324951634142...|  0.0|
|[0.82226325870284...|  0.0|
|[0.84406658980857...|  0.0|
|[0.85177230541965...|  0.0|
|[0.86586281889129

### step 2. split the dataset into training and validation dataset.

In [5]:
val data0 = data.filter($"Class" === "0").cache()
val data1 = data.filter($"Class" === "1").cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

data0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
data1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [6]:
val splitTime0 = data0.stat.approxQuantile("Time", Array(0.7), 0.001).head
val splitTime1 = data1.stat.approxQuantile("Time", Array(0.6), 0.001).head
val trainingData0 = data0.filter(s"Time<$splitTime0").cache()
val validationData0 = data0.filter(s"Time>=$splitTime0").cache()
val trainingData1 = data1.filter(s"Time<$splitTime1").cache()
val validationData1 = data1.filter(s"Time>=$splitTime1").cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitTime0: Double = 132857.0
splitTime1: Double = 93824.0
trainingData0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
trainingData1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [7]:
trainingData0.select("V1", "Class").show(10)
trainingData1.select("V1", "Class").show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+
|                V1|Class|
+------------------+-----+
|  -1.3598071336738|  0.0|
|  1.19185711131486|  0.0|
| -1.35835406159823|  0.0|
|-0.966271711572087|  0.0|
| -1.15823309349523|  0.0|
|-0.425965884412454|  0.0|
|  1.22965763450793|  0.0|
|-0.644269442348146|  0.0|
| -0.89428608220282|  0.0|
| -0.33826175242575|  0.0|
+------------------+-----+
only showing top 10 rows

+-------------------+-----+
|                 V1|Class|
+-------------------+-----+
|   -2.3122265423263|  1.0|
|   -3.0435406239976|  1.0|
|  -2.30334956758553|  1.0|
|  -4.39797444171999|  1.0|
|   1.23423504613468|  1.0|
|0.00843036489558254|  1.0|
| 0.0267792264491516|  1.0|
|  0.329594333318222|  1.0|
|  0.316459000444982|  1.0|
|  0.725645739819857|  1.0|
+-------------------+-----+
only showing top 10 rows



In [8]:
val trainingData = trainingData0.unionAll(trainingData1)
val validationData = validationData0.unionAll(validationData1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [9]:
println(" Training set statistics: 1 represents fraud and 0 represents normal")
trainingData.groupBy("Class").count().show()
println(" validation set statistics: 1 represents fraud and 0 represents normal")
validationData.groupBy("Class").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 Training set statistics: 1 represents fraud and 0 represents normal
+-----+------+
|Class| count|
+-----+------+
|  0.0|198835|
|  1.0|   295|
+-----+------+

 validation set statistics: 1 represents fraud and 0 represents normal
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85480|
|  1.0|  197|
+-----+-----+



### Step3. Buile the Model

In [10]:
//Gradient-boosted tree classifier
// Train a GBT model.
//setImpurity("entropy") 
val t = System.nanoTime
val gbt = new GBTClassifier().setLabelCol("label").setFeaturesCol("features").setMaxIter(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 637978714642377
gbt: org.apache.spark.ml.classification.GBTClassifier = gbtc_ce3a819e59a3


In [11]:
val modelGBT = gbt.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelGBT: org.apache.spark.ml.classification.GBTClassificationModel = GBTClassificationModel (uid=gbtc_ce3a819e59a3) with 10 trees
durationtrain: Double = 27.987315374

initial model training finished.
Training process takes 27.987315374 secs


In [12]:
val s = System.nanoTime
val predictionsGBT = modelGBT.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsGBT.cache()
//val predictionsAndLabel: RDD[Row] = df.rdd= predictionsGBT.select("prediction", "label")
predictionsGBT.select("prediction", "label").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 638008931281050
predictionsGBT: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.244981766

initial model training finished.
Training process takes 0.244981766 secs
res20: predictionsGBT.type = [V1: double, V2: double ... 35 more fields]
+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [13]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsGBT.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85480|
|  1.0|  197|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85536|
|       1.0|  141|
+----------+-----+



In [14]:
println(s"Matrice de confusion :")
predictionsGBT.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85474|
|       1.0|  0.0|    6|
|       0.0|  1.0|   62|
|       1.0|  1.0|  135|
+----------+-----+-----+



In [15]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsGBT)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsGBT))
println("Accuracy = " + evaluator1.evaluate(predictionsGBT))
println("Precision = " + evaluator2.evaluate(predictionsGBT))
println("Recall = " + evaluator3.evaluate(predictionsGBT))
println("F1 = " + evaluator4.evaluate(predictionsGBT))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_71a5c7f69cfb
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_f3eb0494d38f
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_1e7e5c5ccfa6
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_0d3daa77cff1
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_ae5f05011803
accuracy: Double = 0.9992063214164828
Area Under ROC Curve = 0.8426044979797572
Accuracy = 0.9992063214164828
Precision = 0.9991789816680409
Recall = 0.9992063214164829
F1 = 0.9991407028416789
Test Error = 7.936785835171989E-4


In [16]:
// Random forest classifier
// Train a RandomForest model.
val rf = new RandomForestClassifier().setLabelCol("label").setFeaturesCol("features").setNumTrees(10)
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_52a856193b7c
t: Long = 638022232109834


In [17]:
val modelRF = rf.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelRF: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel (uid=rfc_52a856193b7c) with 10 trees
durationtrain: Double = 4.720912063

initial model training finished.
Training process takes 4.720912063 secs


In [18]:
val s = System.nanoTime
val predictionsRF = modelRF.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsRF.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 638028088981162
predictionsRF: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.24319453

initial model training finished.
Training process takes 0.24319453 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.80345987710294...|
|       0.0|  0.0|[0.81607634463416...|
|       0.0|  0.0|[0.83176453231324...|
|       0.0|  0.0|[0.82939280217800...|
|       0.0|  0.0|[0.82856290285761...|
+----------+-----+--------------------+
only showing top 5 rows



In [19]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsRF.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85480|
|  1.0|  197|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85535|
|       1.0|  142|
+----------+-----+



In [20]:
println(s"Matrice de confusion :")
predictionsRF.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85478|
|       1.0|  0.0|    2|
|       0.0|  1.0|   57|
|       1.0|  1.0|  140|
+----------+-----+-----+



In [21]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsRF)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsRF))
println("Accuracy = " + evaluator1.evaluate(predictionsRF))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_0a658aec7435
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_4b7401ff4eb1
accuracy: Double = 0.9993113671113601
Area Under ROC Curve = 0.8553182505956212
Accuracy = 0.9993113671113601
Test Error = 6.886328886398507E-4


In [22]:
// Decision Tree classifier
// Train a DecisionTree model.
val dt = new DecisionTreeClassifier().setLabelCol("label").setFeaturesCol("features")

val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dt: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_e2b54acc5fc7
t: Long = 638035220307465


In [23]:
val modelDT = dt.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelDT: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_e2b54acc5fc7) of depth 5 with 25 nodes
durationtrain: Double = 2.972453781

initial model training finished.
Training process takes 2.972453781 secs


In [24]:
val s = System.nanoTime
val predictionsDT = modelDT.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsDT.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 638039119927914
predictionsDT: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.218285513

initial model training finished.
Training process takes 0.218285513 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.80345987710294...|
|       0.0|  0.0|[0.81607634463416...|
|       0.0|  0.0|[0.83176453231324...|
|       0.0|  0.0|[0.82939280217800...|
|       0.0|  0.0|[0.82856290285761...|
+----------+-----+--------------------+
only showing top 5 rows



In [25]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsDT.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85480|
|  1.0|  197|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85537|
|       1.0|  140|
+----------+-----+



In [26]:
println(s"Matrice de confusion :")
predictionsDT.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85478|
|       1.0|  0.0|    2|
|       0.0|  1.0|   59|
|       1.0|  1.0|  138|
+----------+-----+-----+



In [27]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsDT)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsDT))
println("Accuracy = " + evaluator1.evaluate(predictionsDT))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_5571884cbac3
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_3a2065a298b3
accuracy: Double = 0.9992880236236096
Area Under ROC Curve = 0.8502421084636416
Accuracy = 0.9992880236236096
Test Error = 7.119763763904219E-4


In [28]:
// SVM classifier
// Train a SVM model.
val svm = new LinearSVC().setLabelCol("label").setFeaturesCol("features").setMaxIter(100).setRegParam(0.1) 
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

svm: org.apache.spark.ml.classification.LinearSVC = linearsvc_45441ba97388
t: Long = 638046213964870


In [29]:
val modelSVM = svm.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelSVM: org.apache.spark.ml.classification.LinearSVCModel = linearsvc_45441ba97388
durationtrain: Double = 108.595590178

initial model training finished.
Training process takes 108.595590178 secs


In [30]:
val s = System.nanoTime
val predictionsSVM = modelSVM.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsSVM.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 638155556075998
predictionsSVM: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 34 more fields]
durationprediction: Double = 0.239918081

initial model training finished.
Training process takes 0.239918081 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.80345987710294...|
|       0.0|  0.0|[0.81607634463416...|
|       0.0|  0.0|[0.83176453231324...|
|       0.0|  0.0|[0.82939280217800...|
|       0.0|  0.0|[0.82856290285761...|
+----------+-----+--------------------+
only showing top 5 rows



In [31]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsSVM.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85480|
|  1.0|  197|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85647|
|       1.0|   30|
+----------+-----+



In [32]:
println(s"Matrice de confusion :")
predictionsSVM.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85480|
|       0.0|  1.0|  167|
|       1.0|  1.0|   30|
+----------+-----+-----+



In [33]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsSVM)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsSVM))
println("Accuracy = " + evaluator1.evaluate(predictionsSVM))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_b4e1231f537f
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_ace29a246edb
accuracy: Double = 0.9980508187728329
Area Under ROC Curve = 0.5761421319796954
Accuracy = 0.9980508187728329
Test Error = 0.0019491812271671405


In [34]:
// Logistic Regression classifier
val lr = new LogisticRegression().setLabelCol("label").setFeaturesCol("features").setMaxIter(10).setTol(1E-6).setFitIntercept(true) 
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_c48c9f398ae0
t: Long = 638162680667133


In [35]:
val Modellr = lr.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Modellr: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_c48c9f398ae0, numClasses = 2, numFeatures = 11
durationtrain: Double = 3.947466001

initial model training finished.
Training process takes 3.947466001 secs


In [36]:
val s = System.nanoTime
val predictionsLR = Modellr.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsLR.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 638168608222291
predictionsLR: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.215285272

initial model training finished.
Training process takes 0.215285272 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.80345987710294...|
|       0.0|  0.0|[0.81607634463416...|
|       0.0|  0.0|[0.83176453231324...|
|       0.0|  0.0|[0.82939280217800...|
|       0.0|  0.0|[0.82856290285761...|
+----------+-----+--------------------+
only showing top 5 rows



In [37]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsLR.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85480|
|  1.0|  197|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85607|
|       1.0|   70|
+----------+-----+



In [38]:
println(s"Matrice de confusion :")
predictionsLR.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85480|
|       0.0|  1.0|  127|
|       1.0|  1.0|   70|
+----------+-----+-----+



In [39]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsLR)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsLR))
println("Accuracy = " + evaluator1.evaluate(predictionsLR))
println("Precision = " + evaluator2.evaluate(predictionsLR))
println("Recall = " + evaluator3.evaluate(predictionsLR))
println("F1 = " + evaluator4.evaluate(predictionsLR))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_a3dac9222ad6
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_ec89c6f7d14b
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_2ec72ab5b6f7
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_beaca3eff0f5
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_57c811004651
accuracy: Double = 0.998517688527843
Area Under ROC Curve = 0.6776649746192893
Accuracy = 0.998517688527843
Precision = 0.9985198875718109
Recall = 0.998517688527843
F1 = 0.9981657039460605
Test Error = 0.0014823114721570496


In [40]:
// Naive Bayes classifier
// Train a NaiveBayes model.
val nb = new NaiveBayes().setLabelCol("label").setFeaturesCol("features").setSmoothing(1.0)
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

nb: org.apache.spark.ml.classification.NaiveBayes = nb_3d16f25856b5
t: Long = 638178809553759


In [41]:
/*val scaler = new MinMaxScaler().setInputCol("features").setOutputCol("featuresScaled")
val pipeline = new Pipeline().setStages(Array(scaler))
val pipelineModel1 = pipeline.fit(trainingData)
val pipelineModel2 = pipeline.fit(trainingData)
val datatrain = pipelineModel1.transform(trainingData)
val datatest = pipelineModel2.transform(validationData)*/

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
val modelNB = nb.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelNB: org.apache.spark.ml.classification.NaiveBayesModel = NaiveBayesModel (uid=nb_3d16f25856b5) with 2 classes
durationtrain: Double = 1.504088299

initial model training finished.
Training process takes 1.504088299 secs


In [43]:
val s = System.nanoTime
val predictionsNB = modelNB.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsNB.select("prediction", "label").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 638180927249796
predictionsNB: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.200728699

initial model training finished.
Training process takes 0.200728699 secs
+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [44]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsNB.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85480|
|  1.0|  197|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85677|
+----------+-----+



In [45]:
println(s"Matrice de confusion :")
predictionsNB.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85480|
|       0.0|  1.0|  197|
+----------+-----+-----+



In [46]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsNB)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsNB))
println("Accuracy = " + evaluator1.evaluate(predictionsNB))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_5643a6de63f0
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_d900d41904e6
accuracy: Double = 0.9977006664565753
Area Under ROC Curve = 0.5
Accuracy = 0.9977006664565753
Test Error = 0.0022993335434247086


In [47]:
// Multilayer Perceptron Classifier
// create the trainer and set its parameters
val t = System.nanoTime
val layers = Array[Int] (11, 6, 3, 2)
val mlp = new MultilayerPerceptronClassifier().setLayers(layers).setLabelCol("label").setFeaturesCol("features").setTol(1E-4).setBlockSize(128).setSeed(1234L).setMaxIter(25) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 638187958685713
layers: Array[Int] = Array(11, 6, 3, 2)
mlp: org.apache.spark.ml.classification.MultilayerPerceptronClassifier = mlpc_afef2ebfd571


In [48]:
// Train a Multilayer Perceptron model.
val modelMLP = mlp.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelMLP: org.apache.spark.ml.classification.MultilayerPerceptronClassificationModel = mlpc_afef2ebfd571
durationtrain: Double = 13.247840213

initial model training finished.
Training process takes 13.247840213 secs


In [49]:
val s = System.nanoTime
val predictionsMLP = modelMLP.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsMLP.select("prediction","label").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 638202081230946
predictionsMLP: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.205272327

initial model training finished.
Training process takes 0.205272327 secs
+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [50]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsMLP.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85480|
|  1.0|  197|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85496|
|       1.0|  181|
+----------+-----+



In [51]:
println(s"Matrice de confusion :")
predictionsMLP.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85448|
|       1.0|  0.0|   32|
|       0.0|  1.0|   48|
|       1.0|  1.0|  149|
+----------+-----+-----+



In [52]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsMLP)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsMLP))
println("Accuracy = " + evaluator1.evaluate(predictionsMLP))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_e2f3e6de8039
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_c272d59e9dcf
accuracy: Double = 0.9990662604899798
Area Under ROC Curve = 0.8779854105451687
Accuracy = 0.9990662604899798
Test Error = 9.337395100201817E-4


In [53]:
// One Vs Rest Classifier using Logistic Regression classifier
val classifier = new LogisticRegression().setLabelCol("label").setFeaturesCol("features").setMaxIter(10).setTol(1E-6).setFitIntercept(true) 
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

classifier: org.apache.spark.ml.classification.LogisticRegression = logreg_aa875fc95e45
t: Long = 638210194288264


In [54]:
// train the multiclass model.
val ovr = new OneVsRest().setClassifier(classifier)
val Modelovr = ovr.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ovr: org.apache.spark.ml.classification.OneVsRest = oneVsRest_7661af47fa0c
Modelovr: org.apache.spark.ml.classification.OneVsRestModel = oneVsRest_7661af47fa0c
durationtrain: Double = 7.020285597

initial model training finished.
Training process takes 7.020285597 secs


In [55]:
val s = System.nanoTime
val predictionsOVR = Modelovr.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsOVR.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 638218157228275
predictionsOVR: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 34 more fields]
durationprediction: Double = 0.35484091

initial model training finished.
Training process takes 0.35484091 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.80345987710294...|
|       0.0|  0.0|[0.81607634463416...|
|       0.0|  0.0|[0.83176453231324...|
|       0.0|  0.0|[0.82939280217800...|
|       0.0|  0.0|[0.82856290285761...|
+----------+-----+--------------------+
only showing top 5 rows



In [56]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsOVR.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85480|
|  1.0|  197|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85607|
|       1.0|   70|
+----------+-----+



In [57]:
println(s"Matrice de confusion :")
predictionsOVR.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85480|
|       0.0|  1.0|  127|
|       1.0|  1.0|   70|
+----------+-----+-----+



In [58]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val areaUnderPR = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderPR")
val accuracy = evaluator1.evaluate(predictionsOVR)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsOVR))
println("Area Under the Precision-Recall Curve = "  + areaUnderPR.evaluate(predictionsOVR))
println("Accuracy = " + evaluator1.evaluate(predictionsOVR))
println("Precision = " + evaluator2.evaluate(predictionsOVR))
println("Recall = " + evaluator3.evaluate(predictionsOVR))
println("F1 = " + evaluator4.evaluate(predictionsOVR))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_29f006fcc9a9
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_8f67e19709d5
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_e416e33c32fd
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_00d6d3e3a3c4
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_cc8787edfd0a
areaUnderPR: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_6db7dfa0cd87
accuracy: Double = 0.998517688527843
Area Under ROC Curve = 0.6776649746192893
Area Under the Precision-Recall Curve = 0.6784061303553679
Accuracy = 0.998517688527843
Precision = 0.9985198875718109
Recall = 0.998517688527843
F1 = 0.9981657039460605
Test Error = 0.0014823114721570496
